Inputs: HOEP and Load Data

Decision variables: PCEA PCDDR (24 x 6)

Version 2.0 
        
        PCDDR included for possible revenue earned
        Asset costs and demand charges included

Objective Function 

        Profits = Revenue - Expenses
        
        Profits = (KRDDR*PCDDR) - (NTB + asset)            (1)

Constraints:         
        
        PNT = PDLL + PCEA                                  (2)
        
        MEC = sum(30* (sum(PNT * (HOEP + WMST)))           (3)
        
        PD = PNT + PSB                     (13.5)
        
        PCDDR <= ESB <= 10000              (14)
        
        ESB(t) = PCEA + ESB(t-1) - PCDDR   (14.1)
        
        ESB Constraints
        
        -5000 <= PSB <= 5000               (10)
        
        |PCDDR| + |PCEA| <= 5000           (11)
        5000 - (|PCDDR| + |PCEA|) >= 0
        
        KNC = 8000 lifetime
        
        NC per month = 8000 / 10 years / 12 months = 67 cycles per month

In [1]:
import numpy as np
import pandas as pd
import time
from scipy.optimize import minimize

In [2]:
## Read in csv file from IESO website, ***fill blank entries with 0*** (temp)
df = pd.read_csv("PUB_PriceHOEPPredispOR_2019.csv").fillna(0)

df.rename(columns=df.iloc[2], inplace=True)         ## Set headers to the proper ones row 4
df = df[3:]
df.reset_index(inplace=True, drop=True)             ## Reset indices to proper values

#df.dropna(inplace=True)


## Convert columns to suitable data types
df['Date'] = pd.to_datetime(df['Date'])
df = df.astype({'Hour':int, 'HOEP':float, 'Hour 1 Predispatch': float, 'Hour 2 Predispatch': float, 'Hour 3 Predispatch':float, 'OR 10 Min Sync':float, 'OR 10 Min non-sync':float, 'OR 30 Min':float})

## Split the date into year, month, day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop(['Date'], axis=1, inplace=True)

## Rearrange them so they appear at the beginning (not necessary, only intermediate step for you to visualize)
date = ['Year', 'Month', 'Day']
df = df[date + [c for c in df if c not in date]]

## Create new dataframe for final data values
data = pd.DataFrame()

## Iterate through the months of the year specified from CSV file
## and iterate through the hours to get monthly average for that specific hour
for month in range(1,13):
    average = []
    df_month = df.loc[df['Month'] == month]

    for hour in range(1,25):
        h = df_month.loc[df['Hour'] == hour]
        average.append((h['HOEP'].sum()/h.shape[0])/100)  ## Cents to Dollars
    data['Month ' + str(month)] = average

## Set index to proper hours
data.index = range(1, len(data)+1)
data.index.name = 'Hour'


data_6months = pd.DataFrame()
for i in range(0, 12, 2):
    data_6months[data.columns[i] + ' & ' + data.columns[i+1]] = (data[data.columns[i]] + data[data.columns[i+1]])/2
#data_6months

In [3]:
load_data = pd.read_excel('load_data.xlsx', index_col=0)



In [4]:
## Initialize all constants

ESB = 10000
PSB = 5000

# Revenue
KPDDR = 1200.264/8720       ## $856.436 /kW year

# Monthly Bill 
WMST = 0.003499             ## $/kWh 
KDC = 0.43

# Asset Costs
KPB = 40
PS = 5000 
KEB = 500
ES = 10000

# Cycle Constraint
KNC = 67

In [5]:
## x[0:24] = PCEA
## x[24:48] = ESB
## x[48:72] = PCDDR
start_time = time.time()
def constraint1(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i] - x[48+i-1]
    return x[0:24] + x[24:48] - x[48:72]
    
def constraint2(x):
    for i in range(24):
        if (i == 0):
            x[24] = 0
        else:
            x[24+i] = x[24+i-1] + x[i] - x[48+i-1]
    return ESB - (x[0:24]+ x[24:48] - x[48:72])

def constraint4(x): 
    return 5000 - (abs(x[0:24]) + abs(x[48:72]))

def constraint3(x):
    NC = 0 
    for i in range(24):
        if (i == 0): 
            NC = NC + 0
        else:
            if ((x[24+i] - x[24+i-1]) >= 0):
                NC = NC + (x[24+i] - x[24+i-1])
            else: 
                NC = NC + 0
                
    return KNC - NC/ESB*30

power = ((-PSB, PSB),) * 24
storage = ((0, ESB),) * 24
DDR = ((0,PSB),) * 24

##Objective function
def MEC(x):           #      KPDDR*PCDDR - (PCEA+PDLL)*(HOEP + WMST) - PDF*(GAC+CBR) - KDC*max(PDLL+PCEA)
    return -(
        #Revenue
        KPDDR*sum(30*(sum(np.array([x[48:72]])))) - 
        #Costs
            #MEC
             sum(30*sum((load_data.iloc[:,month].to_numpy() + np.array([x[0:24]])) * 
                        (data_6months.iloc[:,month].to_numpy() + WMST))) - 
            #MDC
            #30*KDC*(PDLL + PCEA)
            30*KDC*np.amax(load_data.iloc[:,month].to_numpy() + np.array([x[0:24]]))
            )


x0 = np.array([np.ones(24), np.ones(24), np.ones(24)])
bounds = (power + storage + DDR)
cons1 = {'type': 'ineq', 'fun': constraint1}
cons2 = {'type': 'ineq', 'fun': constraint2}
cons3 = {'type': 'ineq', 'fun': constraint3}
cons4 = {'type': 'ineq', 'fun': constraint4}
cons = ([cons1, cons2, cons3,cons4])

pcea_solutions = []
esb_solutions = []
ddr_solutions = []

mec = 0 

for month in range(6):
    sol = minimize(MEC, x0, method='SLSQP',
                     bounds=bounds,
                     constraints=cons,
                     options= {'maxiter':150,'disp':True})
    mec = mec + sol.fun
    pcea_solutions.append(sol.x[0:24])
    esb_solutions.append(sol.x[24:48])
    ddr_solutions.append(sol.x[48:72])
      

print(-2*mec)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration limit exceeded    (Exit mode 9)
            Current function value: 254888.00523425505
            Iterations: 151
            Function evaluations: 11253
            Gradient evaluations: 151
Iteration limit exceeded    (Exit mode 9)
            Current function value: 70550.86856483316
            Iterations: 151
            Function evaluations: 11376
            Gradient evaluations: 151
Iteration limit exceeded    (Exit mode 9)
            Current function value: -67604.60571792051
            Iterations: 151
            Function evaluations: 11215
            Gradient evaluations: 151
Iteration limit exceeded    (Exit mode 9)
            Current function value: 142184.28815425668
            Iterations: 151
            Function evaluations: 11208
            Gradient evaluations: 150
Iteration limit exceeded    (Exit mode 9)
            Current function value: 50564.35025326086
            Iterations: 151
            Function evaluations: 11234
            Gradient eval

In [6]:
df_sol = pd.concat([pd.DataFrame(data=pcea_solutions).transpose(), 
                    pd.DataFrame(data=esb_solutions).transpose(), 
                    pd.DataFrame(data=ddr_solutions).transpose()], axis=1)
df_sol

,0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5
0,-1.245008e-09,7.761081e-11,1766.190311,1.954415e-13,-7.147016e-10,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.965980e-13,1766.190311,1.652891e-13,0.000000,4.786580e-07
1,1.663901e+03,1.731331e+03,3284.510385,1.726961e+03,3.076790e+03,1647.878782,1663.901264,1731.330682,1518.320074,1726.961151,3076.789536,1647.878781,0.000000,3.909929e-09,1715.489430,2.367247e+01,1911.644608,2.186872e-06
2,1.661328e+03,1.731656e+03,3246.163208,1.717862e+03,3.059297e+03,1685.320515,3325.229128,3462.986534,3048.993852,3421.150969,4224.442138,3333.199294,0.000000,3.909929e-09,1753.836661,2.367247e+01,1940.702778,2.186872e-06
3,1.646417e+03,1.732742e+03,3268.558830,1.746365e+03,2.811713e+03,1668.730873,4971.646129,5195.728223,4563.716021,5143.843565,5095.451885,5001.930164,0.000000,3.909922e-09,1731.440882,2.367247e+01,2114.589450,2.186871e-06
4,1.669118e+03,1.726269e+03,3263.223369,1.721386e+03,3.082778e+03,1678.311512,6640.763666,6921.997511,6095.498508,6841.556697,6063.640373,6680.241674,0.000000,3.909937e-09,1736.776515,2.366961e+01,1896.457829,2.186872e-06
5,1.679618e+03,1.539001e+03,3154.883094,1.602512e+03,2.969811e+03,1659.879125,8320.381838,8460.998755,7513.605088,8420.399292,7136.993153,8340.120797,0.000000,3.909888e-09,1845.116800,2.366961e+01,2030.189379,2.186871e-06
6,8.398091e+02,-2.448027e+03,3063.254585,8.017457e+02,2.466276e+03,829.939562,9160.190924,6012.972220,8731.742873,9198.475333,7573.079853,9170.060357,0.000000,2.951127e-09,1936.745248,2.209836e-01,2533.723910,4.786578e-07
7,-4.580095e+03,-3.006486e+03,-29.152962,4.008728e+02,2.841426e+03,-1712.658190,4580.095460,3006.486110,6765.844663,9599.127175,7880.782290,7457.402166,0.000000,2.951109e-09,4970.846843,1.651700e-13,722.208659,0.000000e+00
8,-5.364537e+02,-1.415266e+03,2482.414266,2.004353e+02,2.045074e+02,-3728.701111,4043.641761,1591.220374,4277.412086,9799.562445,7363.081033,3728.701055,0.000000,2.951119e-09,2517.585689,1.650300e-13,469.027561,0.000000e+00
9,-1.989068e+03,1.272719e+02,2999.548714,1.002188e+02,-9.389329e+02,-1864.350548,2054.573444,1718.492281,4759.375112,9899.781223,5955.120558,1864.350507,0.000000,8.624194e-09,1969.087236,1.650533e-13,325.778916,0.000000e+00


In [7]:
df_sol.to_csv('SampleChartData.csv')